In [1]:
import numpy as np
from scipy import linalg

# Eigenvalues and Eigenvectors for Symmetric / Hermitian Band Matrix

$$
A \mathbf{x} = \lambda \mathbf{x}, \quad \mathrm{s.t.} \; A:\text{ symmetric or Hermitian matrix} 
$$



```python
eigvals, eigvecs = linalg.eig_banded(band_a_h, lower = False)
eigvals = linalg.eig_banded(band_a_h, lower = False, eigvals_only = True)
```
* ```eigval``` - 1D array (vector)
* ```eigvecs``` - 2D array with each eigvec as a columnvector
* ```banded_a_h``` - banded form (lower or upper - default)
* ```lower = False``` - default is uppder band form
* corresponding Lapack function
  1. sbevd (symmetric matrix)
  2. hbevd (Hermitian matrix)

cf) for generalized egienvalue problems - use general method, no function supported for banded form

## Example 1.

$$
A =\begin{bmatrix}
\colorbox{pink}{$1$} & \colorbox{grey}{$5$} & \colorbox{lightgrey}{$2$} & 0\\
5 & \colorbox{pink}{$2$} & \colorbox{grey}{$5$} & \colorbox{lightgrey}{$2$} \\
2 & 5 & \colorbox{pink}{$3$} & \colorbox{grey}{$5$} \\
0 & 2 & 5 & \colorbox{pink}{$4$} 
\end{bmatrix}
$$

upper form

$$
\begin{bmatrix}
0 & 0 & \colorbox{lightgrey}{$2$} & \colorbox{lightgrey}{$2$}\\
0 & \colorbox{grey}{$5$} & \colorbox{grey}{$5$} & \colorbox{grey}{$5$} \\
\colorbox{pink}{$1$} & \colorbox{pink}{$2$} & \colorbox{pink}{$3$} & \colorbox{pink}{$4$}
\end{bmatrix}
$$

lower form

$$
\begin{bmatrix}
\colorbox{pink}{$1$} & \colorbox{pink}{$2$} & \colorbox{pink}{$3$} & \colorbox{pink}{$4$} \\
0 & \colorbox{grey}{$5$} & \colorbox{grey}{$5$} & \colorbox{grey}{$5$} \\
0 & 0 & \colorbox{lightgrey}{$2$} & \colorbox{lightgrey}{$2$}\\
\end{bmatrix}
$$

In [2]:
# A, upper banded form
A_band_h = np.array([
    [0, 0, 2, 2],
    [0, 5, 5, 5],
    [1, 2, 3, 4],
], dtype=np.float64)

print(A_band_h)

[[0. 0. 2. 2.]
 [0. 5. 5. 5.]
 [1. 2. 3. 4.]]


In [3]:
# eigenvalues, eigenvectors form "banded A"
eigvals, eigvecs = linalg.eig_banded(A_band_h, lower=False)
print(eigvals)
print()
print(eigvecs)

[-4.26200532 -2.22987175  3.95222349 12.53965359]

[[ 0.54585106  0.49026342  0.58943537  0.33801529]
 [-0.73403852 -0.04827646  0.41123929  0.53827417]
 [ 0.39896071 -0.67105283 -0.15802574  0.60460427]
 [-0.06375287  0.55407514 -0.6771086   0.48006276]]


In [4]:
"""
# full matrix
A = np.array([
    [1, 5, 2, 0],
    [5, 2, 5, 2],
    [2, 5, 3, 5],
    [0, 2, 5, 4]
], dtype=float)
"""

""" Reconstructing A from A_band_h """
twos  = A_band_h[0, 2:]
fives = A_band_h[1, 1:]
diagonals = A_band_h[2, :]

A_full = np.diag(diagonals, k=0) + np.diag(fives, k=1) + np.diag(twos, k=2) + np.diag(fives, k=-1) + np.diag(twos, k=-2)
print(A_full)
print()

# Check whether the results from using linalg.eig_banded and linalg.eigh are same
eigvals_full, eigvecs_full = linalg.eigh(A_full)
print(eigvals_full)
print(eigvals)
print(f'Are they same? -> {np.allclose(eigvals, eigvals_full)}')

# Check whether the results from using linalg.eigh and linalg.eig are same
eigvals_full_general, eigvecs_full_general = linalg.eig(A_full)
eigvals_full_general = np.sort(np.real_if_close(eigvals_full_general))
print(eigvals_full_general)
print(f'Are they same? -> {np.allclose(eigvals_full, eigvals_full_general)}')

[[1. 5. 2. 0.]
 [5. 2. 5. 2.]
 [2. 5. 3. 5.]
 [0. 2. 5. 4.]]

[-4.26200532 -2.22987175  3.95222349 12.53965359]
[-4.26200532 -2.22987175  3.95222349 12.53965359]
Are they same? -> True
[-4.26200532 -2.22987175  3.95222349 12.53965359]
Are they same? -> True


In [5]:
# sanity check for educational purpose
# AV = VD
D = np.diag(eigvals, k=0)
V = eigvecs

lhs = A_full @ V
rhs = eigvecs * eigvals # V @ D

#print(f'lhs: {lhs}')
#print(f'rhs: {rhs}')
print(f'Are they same? -> {np.allclose(lhs, rhs)}')

Are they same? -> True


## Example 2. 

$$
A =
\begin{bmatrix}
3 & -1 &        &        &        \\
-1 & 3 & -1     &        &        \\
   & -1 & 3     & \ddots &        \\
   &    & \ddots & \ddots & -1     \\
   &    &        & -1     & 3
\end{bmatrix}_{1000 \times 1000}
$$

$$
\tilde{A} =
\begin{bmatrix}
0 & -1 & -1 & \cdots & -1 \\
3 &  3 &  3 & \cdots &  3
\end{bmatrix}_{2 \times 1000}
$$

In [6]:
n = 1000

diagonals = 3 * np.ones((n,))
off_diag = -1 * np.ones((n-1,))

A_full = np.diag(diagonals, k=0) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1) 
print(A_full)

[[ 3. -1.  0. ...  0.  0.  0.]
 [-1.  3. -1. ...  0.  0.  0.]
 [ 0. -1.  3. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  3. -1.  0.]
 [ 0.  0.  0. ... -1.  3. -1.]
 [ 0.  0.  0. ...  0. -1.  3.]]


In [7]:
zr = np.zeros((1, ))
band_row0 = np.hstack((zr, off_diag))
band_row1 = diagonals
A_band = np.vstack((band_row0, band_row1))
print(A_band)

[[ 0. -1. -1. ... -1. -1. -1.]
 [ 3.  3.  3. ...  3.  3.  3.]]


**Performance Test**

In [8]:
import timeit

In [9]:
start = timeit.default_timer()
eigvals_full, eigvecs_full = linalg.eigh(A_full)
end = timeit.default_timer()

# time check
print(f'linalg.eigh time: {end-start:.6f}')

# Sanity check
# AV
lhs = A_full @ eigvecs_full
# VD
rhs = eigvecs_full * eigvals_full

print(f'AV = VD? -> {np.allclose(lhs, rhs)}')

linalg.eigh time: 0.203984
AV = VD? -> True


In [10]:
start = timeit.default_timer()
eigvals_band, eigvecs_band = linalg.eig_banded(A_band, lower=False)
end = timeit.default_timer()

# time check
print(f'linalg.eigh time taken: {end-start:.6f}')

# Sanity check
# A_full and A_band result
print(f'eigh = eigh_banded? -> {np.allclose(eigvals_full, eigvals_band)}')

linalg.eigh time taken: 0.073870
eigh = eigh_banded? -> True
